# Zadig - A platform on which you can vizualize your eating habits

The main dataset we are using for this project is: https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions

We are going to implement an application that:
- Retrieves all of your past eating recipes
- Analyze what nutriments compose most of the food you consume
- Compare that analysis with the recommended daily intake provided by the NHS: https://www.nhs.uk/live-well/eat-well/what-are-reference-intakes-on-food-labels/
- Provide data vizualizations explaining where you lack the most nutriments
- Explain how that might impact your health
- Create a recommendation system that provides dishes that take into account what you need most and/or your personal preferences 

We are using 7 different nutrional measures in order to assess one's habits

Nutrition information:
- calories (#)
- total fat (PDV), 
- sugar (PDV)
- sodium (PDV)
- protein (PDV)
- saturated fat (PDV)
- carbohydrates (PDV))

In [19]:
import pandas as pd
import string

In [3]:
!ls data

PP_recipes.csv              ingr_map.pkl
PP_users.csv                interactions_test.csv
RAW_interactions.csv        interactions_train.csv
RAW_recipes.csv             interactions_validation.csv


In [5]:
df = pd.read_csv('./data/RAW_recipes.csv')

In [6]:
df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [7]:
df[['nutrition']]

,nutrition
0,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]"
1,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]"
2,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]"
3,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]"
4,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]"
...,...
231632,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]"
231633,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]"
231634,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]"
231635,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]"


In [8]:
df[['nutrition']].iloc[0]

nutrition    [51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]
Name: 0, dtype: object

In [22]:
row = df['nutrition'].iloc[0].translate(str.maketrans('', '', string.punctuation))
row = row.split(' ')
print(len(row))
row

7


['515', '00', '130', '00', '20', '00', '40']

In [23]:
df2 = pd.read_csv('./data/RAW_interactions.csv')

In [24]:
df3 = pd.read_csv('./data/PP_users.csv')

In [25]:
df3.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376
